Process temperature maps and save them to FITS files
---
These are in IDL .sav files, so load them into sunpy and save a copy for easier loading.

In [1]:
from eispy.cube import read
import scipy.io
import numpy as np
import sunpy.map
import astropy.units as u
from pathlib import Path

from astropy.wcs import WCS
from reproject import reproject_interp

from pubtools import solar as solartools
import matplotlib.pyplot as plt

In [2]:
files = {1: {'eis_map': 'data/ARs/eisorig_20080110.fits.gz',
             'temp_sav': 'data/ARs/artemp_20080110.sav',
             'dop_sav': 'data/ARs/doppler_20080110.sav',
             'center': (104.56, -105.73),
            },
         2: {'eis_map': 'data/ARs/eis_l0_20130116_120213.fits',
             'temp_sav': 'data/ARs/artemp_20130116_1.sav',
             'dop_sav': 'data/ARs/doppler_20130116_1.sav',
             'center': (484.27, 182.71),
            },
         3: {'eis_map': 'data/ARs/eis_l0_20130116_145643.fits',
             'temp_sav': 'data/ARs/artemp_20130116_2.sav',
             'dop_sav': 'data/ARs/doppler_20130116_2.sav',
             'center': (10.24, 176.73),
            },
        }

Read in maps, set their WCS, and then save to .fits files

In [8]:
for i in range(1, 4):
    data = files[i]
    eis_map = read(data['eis_map'])
    eis_map = eis_map[eis_map.wavelengths[0]]
    wcs = eis_map.wcs.dropaxis(2)
    wcs.wcs.crpix[1] = (eis_map.data.shape[1] + 1)/ 2
    wcs.wcs.crval[1] = (data['center'][1] * u.arcsec).to_value(u.deg)
    wcs.wcs.crpix[0] = (eis_map.data.shape[2] + 1)/ 2
    wcs.wcs.crval[0] = (data['center'][0] * u.arcsec).to_value(u.deg)
        
    for var in ['temp', 'dop']:
        fname = data[f'{var}_sav']
        sav = scipy.io.readsav(fname, python_dict=True)
        temp_data = sav[var].copy()
        if var == 'temp':
            temp_data[temp_data < 0] = np.nan
            temp_data = 10**temp_data / 1e6
        if var == 'dop':
            temp_data = temp_data[:, ::-1]

        temp_map = sunpy.map.Map(temp_data, wcs)
        temp_map = solartools.set_earth_obs_coord(temp_map)
        temp_map.save(Path(data[f'{var}_sav']).with_suffix('.fits'), overwrite=True)

Set MJD-OBS to 54475.952118 from DATE-OBS'. [astropy.wcs.wcs]
Set MJD-OBS to 56308.501539 from DATE-OBS'. [astropy.wcs.wcs]
Set MJD-OBS to 56308.622720 from DATE-OBS'. [astropy.wcs.wcs]


Rotate 3rd map on to 2nd map, and add them together

In [52]:
from sunpy.coordinates import Helioprojective, RotatedSunFrame, transform_with_sun_center

var = 'doppler'# 'artemp'
map_base = sunpy.map.Map(f'data/ARs/{var}_20130116_1.fits')
to_rotate = sunpy.map.Map(f'data/ARs/{var}_20130116_2.fits')
print(map_base.date)
print(to_rotate.date)

2013-01-16T12:02:13.000
2013-01-16T14:56:43.000


Create a new WCS that has enough pixels to fit both maps

In [53]:
new_meta = map_base.meta
new_meta['naxis1'] = 246
new_meta.pop('keycomments')
new_meta
out_wcs = WCS(new_meta)
out_wcs.heliographic_observer = map_base.reference_coordinate.observer

Reproject maps on to the new WCS

In [55]:
maps_in = [map_base, to_rotate]
maps_out = []
for map_in in maps_in:
    dt = map_base.date - map_in.date
    with transform_with_sun_center():
        arr, _ = reproject_interp(map_in, out_wcs, [512, 236])
    maps_out.append(sunpy.map.Map(arr, out_wcs))
    
tot_map = np.nanmean(np.stack((maps_out[0].data, maps_out[1].data)), axis=0)
tot_map = sunpy.map.Map(tot_map, maps_out[0].wcs)
tot_map.save(f'data/ARs/total_map_{var}.fits', overwrite=True)

<ipython-input-55-a16afc903c86>:9: RuntimeWarning: Mean of empty slice
  tot_map = np.nanmean(np.stack((maps_out[0].data, maps_out[1].data)), axis=0)
